In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.models import load_model


In [4]:
# 데이터 경로 설정
# 데이터 경로 설정
train_data_dir = 'img_train_test(누끼)/train'
validation_data_dir = 'img_train_test(누끼)/val'

# 데이터 증강 및 전처리
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2, # 무작위 확대 축소
    horizontal_flip=False # 좌우 반전 끔
)

validation_datagen = ImageDataGenerator(rescale=1.0/255)

# 데이터 로더 생성
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)

# 사전 학습된 ResNet-50 모델 로드 (가중치는 ImageNet으로 사전 학습됨)
base_model = ResNet50(weights='imagenet', include_top=False)

# 새로운 층 추가
x = base_model.output
x = GlobalAveragePooling2D()(x) # 과적합 방지
x = Dense(1024)(x)
x = BatchNormalization()(x) # 안정성 및 속도 향상
x = Activation('relu')(x)
x = Dropout(0.5)(x) # 과적합 방지

x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
# 출력 층
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# 전체 모델 정의
model = Model(inputs = base_model.input, outputs = predictions)

# 기본 모델의 가중치를 고정
for layer in base_model.layers:
    layer.trainable = False

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

# 기본 모델의 가중치를 고정 해제
# 저희가 가지고 있는 데이터에 더 fit한 모델이 될 수있도록
for layer in base_model.layers:
    layer.trainable = True
    
# 모델 재컴파일
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


# 모델을 저장할 파일명 설정
checkpoint_filepath = './model_data/best_model_v4.keras'

# ModelCheckpoint 콜백 설정
checkpoint = ModelCheckpoint(
    checkpoint_filepath,
    monitor='val_accuracy', 
    save_best_only=True, 
    mode='max',
    verbose=1
)

# 모델 훈련 (미세 조정)
history = model.fit(
    train_generator, # 훈련용 데이터셋을 생성
    steps_per_epoch=train_generator.samples // train_generator.batch_size, 
    # 모든 샘플을 한 번씩 사용
    
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10,
    callbacks=[checkpoint]

Found 5994 images belonging to 8 classes.
Found 749 images belonging to 8 classes.
Epoch 1/10


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


374/374 ━━━━━━━━━━━━━━━━━━━━ 304s 795ms/step - accuracy: 0.1665 - loss: 2.5508 - val_accuracy: 0.2065 - val_loss: 1.9814
Epoch 2/10
  1/374 ━━━━━━━━━━━━━━━━━━━━ 4:46 767ms/step - accuracy: 0.1250 - loss: 3.0786

C:\ProgramData\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


374/374 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.1250 - loss: 3.0786 - val_accuracy: 0.2308 - val_loss: 1.9430
Epoch 3/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 311s 827ms/step - accuracy: 0.1775 - loss: 2.3505 - val_accuracy: 0.2011 - val_loss: 2.0565
Epoch 4/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1250 - loss: 2.1246 - val_accuracy: 0.3077 - val_loss: 1.8860
Epoch 5/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 306s 815ms/step - accuracy: 0.2099 - loss: 2.2478 - val_accuracy: 0.2283 - val_loss: 2.1959
Epoch 6/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1875 - loss: 2.5865 - val_accuracy: 0.5385 - val_loss: 1.9432
Epoch 7/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 309s 822ms/step - accuracy: 0.2143 - loss: 2.1990 - val_accuracy: 0.2255 - val_loss: 2.1240
Epoch 8/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1250 - loss: 2.2207 - val_accuracy: 0.1538 - val_loss: 2.0039
Epoch 9/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 302s 805ms/step - accuracy: 0.2240 - loss: 2.1298 - val_accurac

# 성능 평가

In [6]:
# 저장된 모델 로드
saved_model = load_model('./model_data/best_model_v4.keras')

# 테스트 데이터셋 경로 설정
test_data_dir = 'img_train_test(누끼)/test'

# 테스트 데이터셋에 대한 ImageDataGenerator 생성
test_datagen = ImageDataGenerator(rescale=1.0/255)

# 테스트 데이터셋 로드
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)

# 모델 평가
evaluation = saved_model.evaluate(test_generator)

print("Test Accuracy:", evaluation[1])

Found 750 images belonging to 8 classes.
47/47 ━━━━━━━━━━━━━━━━━━━━ 36s 716ms/step - accuracy: 0.6121 - loss: 1.0354
Test Accuracy: 0.6306666731834412


# 예측

In [11]:
from keras.preprocessing import image
import numpy as np
from keras.models import load_model

# 예측하려는 이미지 경로
img1_path = "./image_nuggi(누끼)/men/american/americancasual_25861.jpg"
# image_path = './누끼.jpg'

# 모델 물러오기
test_model = load_model("./model_data/best_model_v4.keras")

# 이미지 불러오기 및 전처리
img = image.load_img(img1_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # 이미지를 0과 1 사이로 정규화 (훈련할 때와 동일한 전처리)

# 예측 생성
predictions = test_model.predict(img_array)

# 결과 해석
predicted_class_index = np.argmax(predictions)

# class_labels = validation_generator.class_indices
class_labels = {'american': 0,    # 이부분만 바꿔줌
 'casual': 1,
 'chic': 2,
 'dandy': 3,
 'formal': 4,
 'gorpcore': 5,
 'sports': 6,
 'street': 7}
predicted_class_label = list(class_labels.keys())[list(class_labels.values()).index(predicted_class_index)]

print("Predicted class:", predicted_class_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted class: american
